In [ ]:
!pip uninstall -y opencv-python opencv-python-headless
!pip install opencv-python==4.5.5.64 opencv-python-headless==4.5.5.64

In [ ]:
# Step 1: Install Required Libraries
!pip install -q numpy

In [ ]:
# Step 2: Download YOLO Files
!wget -q https://pjreddie.com/media/files/yolov3.weights
!wget -q https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
!wget -q https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

In [ ]:
# Step 3: Import Necessary Libraries
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
import base64
from google.colab import output

In [ ]:
# Step 4: Load YOLO Model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
print("✅ YOLO Model Loaded Successfully!")

In [ ]:
# Step 5: JavaScript Code for Capturing Single Frame from Webcam
def video_stream():
    js_code = """
    async function captureFrame() {
        const video = document.createElement('video');
        video.style.display = 'none';
        document.body.appendChild(video);

        const stream = await navigator.mediaDevices.getUserMedia({ video: true });
        video.srcObject = stream;
        await video.play();

        const canvas = document.createElement('canvas');
        const ctx = canvas.getContext('2d');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        ctx.drawImage(video, 0, 0);
        const frame = canvas.toDataURL('image/jpeg', 0.8);
        google.colab.kernel.invokeFunction('notebook.processFrame', [frame], {});
        stream.getTracks().forEach(track => track.stop());
    }
    captureFrame();
    """
    display(Javascript(js_code))

In [ ]:
# Step 6: Decode Base64 Image to OpenCV Format
def decode_frame(data):
    header, encoded = data.split(",", 1)
    img_bytes = base64.b64decode(encoded)
    img_np = np.frombuffer(img_bytes, dtype=np.uint8)
    img = cv2.imdecode(img_np, cv2.IMREAD_COLOR)
    return img

In [ ]:
# Step 7: YOLO Object Detection Function
def detect_objects(img):
    height, width, _ = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids, confidences, boxes = [], [], []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = f"{classes[class_ids[i]]} ({confidences[i]:.2f})"
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return img

In [ ]:
# Step 8: Process Single Frame
def process_frame(data):
    img = decode_frame(data)
    img = detect_objects(img)
    cv2_imshow(img)

In [ ]:
# Step 9: Register Callback and Start Capture
output.register_callback('notebook.processFrame', process_frame)
video_stream()